Ονοματεπώνυμο: Ζαμάγιας Μιχαήλ Ανάργυρος

ΑΜ: ΤΠ5000

## Import modules

In [1]:
import pandas as pd

# Data pre-processing

## Read datasets

In [2]:
datasets = ['LungTrain.txt', 'LungTest.txt', 'KEGG_DATA.txt']
training_dataset, test_dataset, kegg_dataset = [
    pd.read_csv(dataset, sep='\t')
    for dataset in datasets
]

In [3]:
training_genes = training_dataset.iloc[:, 0]

In [4]:
training_columns = list(training_dataset.columns)

In [5]:
training_labels = [
    'Cancer' if 'Cancer' in column else 'Normal' if 'Normal' in column else column
    for column in training_columns
]

In [6]:
training_dataset.set_axis(training_labels, axis='columns')

,Gene,Cancer,Normal,Cancer,Normal,Cancer,Cancer,Cancer,Normal,Cancer,...,Cancer,Normal,Cancer,Normal,Cancer,Cancer,Cancer,Normal,Cancer,Cancer
0,1007_s_at,10.93,10.42,10.63,10.15,10.99,10.78,10.57,10.48,10.55,...,10.35,10.38,10.90,10.24,10.32,10.26,10.50,10.39,11.45,11.75
1,1053_at,6.90,6.92,7.55,6.70,6.83,6.72,6.74,6.72,7.10,...,7.28,6.58,6.80,6.62,7.01,7.24,7.51,6.83,7.68,6.82
2,117_at,8.11,7.76,7.97,7.71,7.78,7.78,7.89,8.06,8.01,...,7.59,7.57,7.88,8.36,8.45,7.92,7.94,7.95,7.72,7.94
3,121_at,9.45,9.52,9.81,9.52,9.86,9.86,10.13,9.88,10.11,...,9.71,9.80,9.84,9.99,10.15,10.12,10.30,10.04,9.83,9.90
4,1255_g_at,4.81,4.72,4.91,4.82,4.82,4.85,5.13,5.04,4.89,...,4.91,4.80,4.88,4.96,4.97,4.95,4.89,4.75,4.90,4.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22278,AFFX-ThrX-5_at,4.97,5.05,5.40,5.14,5.15,5.20,5.49,5.26,5.44,...,5.12,5.19,5.30,5.24,5.24,5.26,5.29,5.21,5.28,5.34
22279,AFFX-ThrX-M_at,4.45,4.46,4.64,4.60,4.58,4.53,4.75,4.49,4.65,...,4.42,4.56,4.61,4.65,4.48,4.56,4.60,4.43,4.63,4.68
22280,AFFX-TrpnX-3_at,4.03,4.04,4.07,3.99,4.07,3.99,4.16,4.14,4.07,...,4.05,4.01,4.12,4.10,4.07,4.08,4.23,4.14,4.16,4.13
22281,AFFX-TrpnX-5_at,4.50,4.54,4.70,4.61,4.63,4.59,4.76,4.61,4.70,...,4.66,4.65,4.77,4.63,4.55,4.67,4.70,4.58,4.82,4.83
